In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import precision_recall_fscore_support
import tensorflow as tf
import numpy as np
import re

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
dossier='/content/drive/MyDrive/'
df=pd.read_csv(dossier+'text_preprocessed_boubacar.csv')
df=df.astype(str).sample(df.shape[0]//150)
df_questions=df["Title"]
df_answers=df["Answers"]

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.head()


,Unnamed: 0,Title,Content,Answers
123181,123181,python - display image web ?,"using python make `` web album viewer '' , mee...","list contains decimal numbers , however exampl..."
120962,120962,python screenshot 2+ monitors ( windows ),"make screenshot python , connected multiple mo...",client sends string `` abcd '' half closes soc...
211809,211809,python-sympy : lambdify returns wrong answer f...,baffled answer following code 10 1. someone he...,xml document . want pull text .. < .p > tags ....
12658,12658,python error using urllib.open,run : import urllib feed = urllib.urlopen ( ``...,"large infrastructure written python 2.6 , rece..."
262748,262748,programatically exiting python script multithr...,"code runs routinely , every ( like month ) pro...",stumbled upon pure python implementation calcu...


In [ ]:
questions_train, questions_test, answers_train, answers_test = train_test_split(df_questions,df_answers, test_size=0.2, random_state=42)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential

# Créer un tokenizer pour convertir les phrases en séquences de mots
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions_train)

# Convertir les phrases en séquences de mots
questions_train = tokenizer.texts_to_sequences(questions_train)
questions_test = tokenizer.texts_to_sequences(questions_test)


# Créer un tokenizer pour convertir les phrases en séquences de mots
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(answers_train)

# Convertir les phrases en séquences de mots
answers_train = tokenizer.texts_to_sequences(answers_train)
answers_test = tokenizer.texts_to_sequences(answers_test)

max_length_questions = max([len(s) for s in questions_train])
max_length_answers = max([len(s) for s in answers_train])
max_length=max([max_length_questions, max_length_answers])
# Pad les séquences pour qu'elles aient toutes la même longueur

questions_train = tf.keras.preprocessing.sequence.pad_sequences(questions_train, maxlen=max_length)
questions_test = tf.keras.preprocessing.sequence.pad_sequences(questions_test, maxlen=max_length)


# Pad les séquences pour qu'elles aient toutes la même longueur

answers_train = tf.keras.preprocessing.sequence.pad_sequences(answers_train, maxlen=max_length)
answers_test = tf.keras.preprocessing.sequence.pad_sequences(answers_test, maxlen=max_length)

# Créer un encoder-decoder avec un LSTM
encoder_inputs = Input(shape=(max_length,1))
encoder = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_length_questions)(encoder_inputs)
encoder = LSTM(64, return_state=True, return_sequences=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_length,))
decoder_embedding = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_length)(decoder_inputs)
decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(len(tokenizer.word_index)+1, activation='softmax')
outputs = decoder_dense(decoder_outputs)


In [ ]:

# Compiler le modèle
model = Model([encoder_inputs, decoder_inputs], [outputs])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Entraîner le modèle
model.fit([questions_train, questions_train], answers_train, epochs=10, batch_size=8, validation_data=([questions_test, answers_test], answers_test), steps_per_epoch=2, validation_steps=2)


Epoch 1/10
2/2 [==============================] - 35s 21s/step - loss: 9.7833 - accuracy: 0.0000e+00 - val_loss: 9.7755 - val_accuracy: 0.9566
Epoch 2/10
2/2 [==============================] - 26s 18s/step - loss: 9.7728 - accuracy: 0.9316 - val_loss: 9.7622 - val_accuracy: 0.9591
Epoch 3/10
2/2 [==============================] - 31s 18s/step - loss: 9.7577 - accuracy: 0.9630 - val_loss: 9.7420 - val_accuracy: 0.9609
Epoch 4/10
2/2 [==============================] - 26s 18s/step - loss: 9.7341 - accuracy: 0.9667 - val_loss: 9.7067 - val_accuracy: 0.9621
Epoch 5/10
2/2 [==============================] - 27s 19s/step - loss: 9.6911 - accuracy: 0.9734 - val_loss: 9.6361 - val_accuracy: 0.9631
Epoch 6/10
2/2 [==============================] - 26s 18s/step - loss: 9.6049 - accuracy: 0.9629 - val_loss: 9.4881 - val_accuracy: 0.9635
Epoch 7/10
2/2 [==============================] - 26s 18s/step - loss: 9.4311 - accuracy: 0.9701 - val_loss: 9.2599 - val_accuracy: 0.9637
Epoch 8/10
2/2 [=======

In [ ]:
# Évaluer le modèle sur les données de test
loss, acc = model.evaluate([questions_test, questions_test], answers_test, verbose=0, batch_size=8)
print("Test accuracy:", acc)

Test accuracy: 0.9659793972969055


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing,metrics,manifold
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,cross_val_predict
from imblearn.over_sampling import ADASYN,SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
import collections
import matplotlib.patches as mpatches
from sklearn.metrics import accuracy_score
%matplotlib inline
from sklearn.preprocessing import RobustScaler
import xgboost
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import classification_report,roc_auc_score,roc_curve,r2_score,recall_score,confusion_matrix,precision_recall_curve
from collections import Counter
from sklearn.model_selection import StratifiedKFold,KFold,StratifiedShuffleSplit
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD,SparsePCA
from sklearn.metrics import classification_report,confusion_matrix
from nltk.tokenize import word_tokenize
from collections import defaultdict
from collections import Counter
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import nltk
from nltk.corpus import stopwords
import string
from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [ ]:
from nltk.stem import *
def stem_traincorpus(data):
    stemmer = PorterStemmer()
    out_data=""
    for words in data:
        out_data+= stemmer.stem(words)
    return out_data
def remove_stopwords(data):
  text=data.lower()
  text_tokens = word_tokenize(text)
  tokens_without_sw = [word for word in text_tokens if not word in stop_words]
  return (" ").join(tokens_without_sw)

In [ ]:
import keras
#def prediction
def predict_text(#define input text and model
                 input_text, tokenizer, model, 
                 #define tokenizer maximum length of sequence
                 maxlen_seq, padding = 'post', truncating = 'post'
                 ):
    
    #prediction
    text = str(input_text)
    sequence = tokenizer.texts_to_sequences([text])
    sequence = pad_sequences(sequence, maxlen = maxlen_seq)
    predict = model.predict([sequence,sequence])
    
    return predict
tokenizer = tf.keras.preprocessing.text.Tokenizer()
new_text=predict_text("print message screen",tokenizer,model,max_length)
print(new_text)
a=tokenizer.sequences_to_texts([new_text])
#tableau=np.array([questions_train, answers_train])
#tableau.reshape(1,tableau.shape[1],tableau.shape[2])

1/1 [==============================] - 0s 342ms/step
[[[6.4416396e-05 6.2272818e-05 6.2088984e-05 ... 5.6042227e-05
   5.6085068e-05 5.5862856e-05]
  [6.8022717e-05 6.4813968e-05 6.4376924e-05 ... 5.6124220e-05
   5.6060366e-05 5.5790140e-05]
  [7.2230439e-05 6.7727771e-05 6.6965811e-05 ... 5.6209959e-05
   5.6037028e-05 5.5711065e-05]
  ...
  [2.1312956e-04 1.4995305e-04 1.3753820e-04 ... 5.6410256e-05
   5.5155899e-05 5.3833661e-05]
  [2.1312956e-04 1.4995305e-04 1.3753820e-04 ... 5.6410256e-05
   5.5155899e-05 5.3833661e-05]
  [2.1312956e-04 1.4995305e-04 1.3753820e-04 ... 5.6410256e-05
   5.5155899e-05 5.3833661e-05]]]


TypeError: ignored